![](https://drive.google.com/uc?export=view&id=1fefljRA_AjosOAts0bK9fGWYU4QiOvUT)

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive/', force_remount=True)
%cd "/content/gdrive/MyDrive/M2 SETSIS/Deep Learning Son/repertoire_tp/3.RealtimeEvaluation"
!echo $PWD

Mounted at /content/gdrive/
/content/gdrive/MyDrive/M2 SETSIS/Deep Learning Son/repertoire_tp/3.RealtimeEvaluation
/content/gdrive/MyDrive/M2 SETSIS/Deep Learning Son/repertoire_tp/3.RealtimeEvaluation


<h1 align="left" style="color:#000051;font-size: 30px">TP : Classifiez des émotions vocales avec du deep learning</h1>

Pour ce TP, vous implémenterez un classifieur à base de réseau de neurones sur des données combinant les jeux de données RAVDESS et TESS avec la librairie keras. 
Repartez de ce notebook Jupyter qui passe en revue chaque partie du TP pour présenter des preuves et une analyse de vos résultats

<h1 align="left" style="color:#000051;font-size: 25px">Partie 3 : Evaluation en temps réel</h1>

Une fois le modèle entrainé et évalué, nous allons l'utiliser en temps réel pour prédire l'émotion dans la voix d'un locuteur. Dans cette partie, il est necessaire de disposer d'un microphone microphone USB ou microphone intégré de votre ordinateur)

<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span>💾 <strong>A propos des jeux de données </strong></span></h2>

Pour ce TP, nous utiliserons 2 jeux de données

- **RAVDESS : Ryerson Audio-Visual Database of Emotional Speech and Song**
https://zenodo.org/record/1188976#.X4sE0tDXKUl
  - RAVDESS a été enregistrée avec 24 acteurs professionnels (12 femmes, 12 hommes), prononçant deux phrases lexicalement identiques avec un accent nord-américain neutre. Chaque phrase est prononcée avec deux niveaux d'intensité émotionnelle (normal, fort).
  - **1440 fichiers** = 24 acteurs x 60 fichiers audio par acteur
  - **8 émotions** (neutre, calme, joie, tristesse, colère, peur, dégout, surprise).



- **TESS : Toronto Emotional Speech Set**
https://tspace.library.utoronto.ca/handle/1807/24487
  - Ces données ont été enregistrées par le Northwestern University Auditory. Un ensemble de 200 mots cibles ont été prononcés dans la phrase "Dites le mot _____" par deux actrices (âgées de 26 et 64 ans) et des enregistrements ont été réalisés lorsque ces phrases ont été prononcées avec chacune des sept émotions décrites ci-dessous.
Les deux actrices ont été recrutées dans la région de Toronto. Les deux actrices parlent l'anglais comme première langue, ont fait des études universitaires et ont une formation musicale.
  - **2800 fichiers** = 2 acteurs x 200 phrases x 7 émotions
  - **7 émotions** (neutre, joie, tristesse, colère, peur, dégoût, surprise)('calme' ne fait pas partie de cette BD) 

Pour ce TP, pour des considérations de volume de données, nous n'avons retenu que 4 des émotions:
- **neutre**
- **joie**
- **tristesse**
- **colère**

<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span>📥 <strong>1. Import des librairies </strong></span></h2>

Si vous avez besoin d'installer des libraires Python pour ce TP, décommentez et exécutez la cellule ci-dessous

In [ ]:
#!pip3 install numpy==1.18.5
#!pip3 install pydub
#!pip3 install librosa
#!pip3 install noisereduce
#!pip3 install matplotlib
#!pip3 install IPython
#!pip3 install tensorflow
#!pip3 install scikit-learn
#!pip3 install scipy
#!pip3 install pyaudio
#!pip3 install wave

In [ ]:
# Librairies de la bibliothèque standard Python
import os
import random
import sys
import time

# Librairies de calcul numérique
import numpy as np

# Librairies de traitement audio
from pydub import AudioSegment, effects
import librosa
import soundfile as sf
from scipy.io import wavfile
from scipy.io.wavfile import write
import noisereduce as nr
from librosa import display   
import pyaudio
import wave

# Librairies de data-visualization
import matplotlib.pyplot as plt
import IPython.display as ipd 
import seaborn as sns
%matplotlib inline

# Librairies de machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
import time
import tensorflow
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Permute, Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D, Bidirectional, Conv2D
from tensorflow.keras.layers import GRU, CuDNNGRU, LSTM, Bidirectional, BatchNormalization, Reshape, MaxPooling2D, Flatten
from tensorflow.keras.utils import normalize, plot_model
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span>📥 <strong>2. Import des fonctions de pré-traitements </strong></span></h2>

Importez ici la fonction **"process_data()"** permettant de réaliser la chaine de pré-traitement sur un fichier audio

In [ ]:
from preprocessing import process_data

In [ ]:
EMOTION_DICT = {0:"neutre", 1:"joie", 2:"tristesse", 3:"colere"}

In [ ]:
FILE = "../1.DataPreprocessing/tmp.wav"
sampling_rate = librosa.core.get_samplerate(FILE)

process_data(audio_filename=FILE, 
             hop_length=512, 
             frame_length=2048, 
             sampling_rate=sampling_rate)

<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span>🧠 <strong>3. Chargement du modèle de deep learning entrainé</strong></span></h2>

Nous allons utiliser le modèle que nous avons entrainé et sauvegardé précédemment. Nous allons donc charger ce modèle avec **load_model** de Keras

In [ ]:
MODEL_FILE = "../2.ModelTraining/model.h5"
from tensorflow.keras.models import load_model

model = load_model(MODEL_FILE)

<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span>🔎 <strong>4. Fonction de prédiction d'une émotion à partir d'un fichier audio de 96000 échantillons</strong></span></h2>

Lors de l'étape de pré-traitement, nous avons égaliser tous les fichiers sonores afin qu'ils soient tous de longueur 96000 échantillons. Cette égalisation permet d'avoir les mêmes dimensions de données en entrée du modèle. 

Nous allons ici développer une fonction qui permet de bout-en-bout de prendre en entrée des fichiers audio de 96000 échantillons et prédire en sortie l'émotion exprimée dans le fichier audio.

<p style="text-align: left; font-size: 16px; color:#131fcf"><span>🖥️  Ecrivez le code dans la fonction  <strong>detect_emotion_from_2sfile</strong> permettant de détecter l'émotion dans un fichier audio de 2 secondes environ (96000 échantillons) à partir d'un fichier audio en entrée, puis tester cette fonction sur le fichier exemple <strong>FILE</strong></span></p>

<p style="text-align: left; font-size: 16px; color:#131fcf"><span>🖥️  Des emojis sont disponibles dans le dossier <strong>"assets"</strong>. </span></p>

In [ ]:
HOP_LENGTH = 512
FRAME_LENGTH = 2048

In [ ]:
def detect_emotion_from_2sfile(audio_filename, display_emoji=True, display_chart=True):
    
    '''
    Fonction permettant de détecter l'émotion dans un fichier audio de 2 secondes environ (96000 échantillons)

            Parameters:
                    audio_filename (str): Chaine de caractère correspondant au chemin d'accès au fichier audio
                    display_emoji (bool) : Booléen indiquant d'afficher ou non l'émoji correspondant à l'émotion détectée
                    display_chart (bool): Booléen indiquant d'afficher ou non un graphique de répartition de probabilité d'appartenance aux 4 émotions du jeu de donnnées


            Returns:
                    preprocessed_array (Numpy array): Matrice de nombres représentant les échantillons audio après pré-traitement (normalisation, ...)
                    emotion_name (str): Chaine de caractères indiquant le nom de l'émotion détectée


    '''
    
    sampling_rate = librosa.core.get_samplerate(audio_filename)

    preprocessed_array, feature = process_data(audio_filename = audio_filename, 
                                               hop_length     = HOP_LENGTH, 
                                               frame_length   = FRAME_LENGTH, 
                                               sampling_rate  = sampling_rate)
    
    feature = normalize(feature)
    
    feature = np.expand_dims(feature, axis=0)
    
    predictions_proba = model.predict_proba(feature)
    
    emotion_name = EMOTION_DICT[np.argmax(predictions_proba)]
    if display_emoji:
        ipd.display(ipd.Image("./assets/{}.png".format(emotion_name),width=30, height=30))
    
    if display_chart:    
        fig = plt.figure(figsize = (10, 2))
        plt.bar(list(EMOTION_DICT.values()), predictions_proba.tolist()[0], color = 'darkturquoise')
        plt.ylabel("Probabilty (%)")
        plt.show()
    
    return preprocessed_array, emotion_name

In [ ]:
FILE = "../1.DataPreprocessing/dataset/tess/OAF_yes_angry.wav"


<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span>🎤 <strong>5. Fonction d'acquisition des données audio en temps réel à partir d'un microphone</strong></span></h2>

Le but de cette partie consiste à développer une fonction qui capture en temps réel le flux audio d'un microphone puis enregistrer séquentiellement ce flux en fichier audio wav temporaires de 96000 échantillons (2 secondes avec fréquence d'échantillonnage à 48 KHz)

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span>🎤 <strong>5.1 Découverte des périphériques audio disponibles </strong></span></h3>

Avant d'utiliser un microphone comme périphérique d'entrée, visualisons l'ensemble des périphériques connéctés à l'ordinateur ainsi que quelques une de leurs caractéristiques

<p style="text-align: left; font-size: 16px; color:#131fcf"><span>🖥️  Executez la fonction  <strong>discover_audio_devices</strong> permettant d'afficher les périphériques audio mono ou stéréo connectés à l'ordinateur. Répérez l'identifiant correspondant au périphérique que vous voulez utiliser </span></p>

In [ ]:
def discover_audio_devices():
    
    '''
    Fonction permettant d'afficher les périphériques audio mono ou stéréo connectés à l'ordinateur

            Parameters:
                    Rien : affiche la liste des péripĥériques
                    
            Returns:
                    Rien : affiche la liste des péripĥériques


    '''
        
    p = pyaudio.PyAudio()
    info = p.get_host_api_info_by_index(0)
    numdevices = info.get('deviceCount')
    device_index = None

    for i in range(0, numdevices):
        channels = p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')
        if  channels == 1 or channels == 2:
            DeviceName = p.get_device_info_by_host_api_device_index(0, i).get('name')
            print("ID : {} ==> {} ==> Nombre de canaux = {}".format(i, DeviceName, channels))
 
            supported_samplerates = []
            SAMPLERATES = [16000, 32000, 44100, 48000]
            devinfo = p.get_device_info_by_index(i)


            for fs in SAMPLERATES:
                try:
                    p.is_format_supported(fs,  # Sample rate
                                          input_device=devinfo["index"],
                                          input_channels=devinfo['maxInputChannels'],
                                          input_format=pyaudio.paInt32)
                except Exception as e:
                    pass
                    #print(fs, e)
                else:
                    supported_samplerates.append(fs)

            print("Fréquence d'échantillonnage supportés = ", supported_samplerates)

In [ ]:
discover_audio_devices()

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span>🎤 <strong>5.2 Développement de la fonction d'acquisition temps réel </strong></span></h3>

Dans cette partie, nous allons exploiter le périphérique audio détecté précédemment pour enregsitrer en boucle un fichier temporaire "tmp.wav" de 2 secondes à 48000 Hz (soit 96000 échantillons).

Pour faire l'acquisition des 96000 échantillons, nous allons utiliser la librairie PyAudio qui permet de créer des objets "stream". 

L'acquisition est itératif, c'est-à-dire que les échantillons sont acquis lot par lot. Les lots s'appellent des **"chunks"**

<p style="text-align: left; font-size: 16px; color:#131fcf"><span>🖥️  Ecrivez le code la fonction  <strong>record_2swav_from_stream</strong> d'enregistrer des fichiers audio de 2 secondes environ (96000 échantillons) à partir d'un périphérique audio d'entrée. Testez ensuite la fonction sur un de vos périphériques audio d'entrée. </span></p>

<p style="text-align: left; font-size: 16px; color:#ec8f1a"><span>📚  
PyAudio : </span> <a href="http://people.csail.mit.edu/hubert/pyaudio/#examples">http://people.csail.mit.edu/hubert/pyaudio/#examples

<p style="text-align: left; font-size: 16px; color:#ec8f1a"><span>📚  
Reading realtime audio data into numpy array
 : </span> <a href="https://stackify.dev/200442-reading-realtime-audio-data-into-numpy-array">https://stackify.dev/200442-reading-realtime-audio-data-into-numpy-array

<p style="text-align: left; font-size: 16px; color:#ec8f1a"><span>📚  
scipy.io.wavfile.write
 : </span> <a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.wavfile.write.html">https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.wavfile.write.html

In [ ]:
# Initialize variables
SAMPLING_RATE = 48000
CHUNK = 512
RECORD_SECONDS = 2

FORMAT = pyaudio.paInt32
CHANNELS = 1

# Open an input channel
p = pyaudio.PyAudio()

In [ ]:
def record_2swav_from_stream(stream, chunk, sampling_rate, record_seconds, save_wav=True):
    
    '''
    Fonction permettant d'enregistrer des fichiers audio de 2 secondes environ (96000 échantillons) à partir d'un périphérique audio d'entrée

            Parameters:
                    stream (Objet Pyaudio stream): Objet Pyaudio stream correspondant au périphérique audio à utiliser
                    chunk (int) : Nombre entier représentant la taille du groupe d'échantillon à acquérir à  chaque itération
                    record_seconds (int) : Nombre entier la durée en secondes du fichier audio final
                    sampling_rate (int) : Nombre entier représentant la fréquence d'échantillonnage du fichier audio


            Returns:
                    full_frames (Numpy array): Matrice de nombres représentant les échantillons audio du fichier audio final de 2 secondes (96000 échantillons)


    '''
    
    return full_frames

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span>🎤 <strong>5.3 Développement d'une fonction de détection de silence </strong></span></h3>

Nous développons ici une fonction de détection de silence. Le but de cette fonction est de faire l'acquisition en boucle de fichiers audio, puis lorsque pendant 5 secondes le niveau sonore est inférieur à un seuil, arrêter l'acquisition.

<p style="text-align: left; font-size: 16px; color:#131fcf"><span>🖥️  Exécutez le code ci-dessous pour tester la fonctionnalité de détection de silence </span></p>

In [ ]:
def is_silent(data):
    # Returns 'True' if below the 'silent' threshold
    return max(data) < 100

In [ ]:
SILENT_SECONDS = 5

In [ ]:
SAMPLES_SILENT = SILENT_SECONDS*SAMPLING_RATE
silent_check_array = np.zeros(SAMPLES_SILENT)

while True:
    t0 = time.time()
    data_array = record_2swav_from_stream(stream=stream, 
                                        chunk=CHUNK, 
                                        sampling_rate=SAMPLING_RATE, 
                                        record_seconds=RECORD_SECONDS,
                                        save_wav=False)
    
    librosa.display.waveplot(data_array.astype(float), SAMPLING_RATE)
    plt.show()
    
    print(time.time()-t0)
    silent_check_array = np.append(silent_check_array, data_array)
    silent_check_array = silent_check_array[-SAMPLES_SILENT:]
    print("MAX => ", np.max(silent_check_array))
    
    if is_silent(data=silent_check_array):
        print("[INFO] : Silence détecté ... Arrêt de l'acquisition audio !")
        break

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span>🗂️ <strong>6 Développement de la fonction de détection en temps réel à partir d'un fichier audio</strong></span></h3>

Le but ici est d'analyser les émotions exprimées dans un fichier audio à partir des fonctions développées précédemment.

Le procédé consiste à charger la totalité des échantillons du fichier audio, puis découper par lot de 96000 échantillons, enregistrer dans un fichier temporaire, et exécuter la détection d'émotion sur le fichier temporaire

In [ ]:
TEST_AUDIO_FILE = "./greta_thunberg.wav"
MAX_SAMPLES = 96000

<p style="text-align: left; font-size: 16px; color:#131fcf"><span>🖥️  Ecrivez le code permettant de détecter les émotions présente dans le fichier <strong>TEST_AUDIO_FILE</strong> </span></p>

<p style="text-align: left; font-size: 16px; color:#ec8f1a"><span>📚  
librosa.load : </span> <a href="https://librosa.org/doc/main/generated/librosa.load.html">https://librosa.org/doc/main/generated/librosa.load.html

<p style="text-align: left; font-size: 16px; color:#ec8f1a"><span>📚  
scipy.io.wavfile.write
 : </span> <a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.wavfile.write.html">https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.wavfile.write.html

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span>🎤 <strong>5.5 Développement de la fonction de détection en temps réel à partir d'un microphone</strong></span></h3>

Le but ici est d'analyser les émotions exprimées directement dans un flux audio en provenance d'un microphone.

Le procédé consiste à faire l'acquisition des échantillons audio chunk par chunk, puis chaque fois que 96000 échantillons sont acquis, ils sont enregistrés sous forme d'un fichier temporaire. Le fichier temporaire est ensuite analysé en utilisant la fonction de détection d'émotion.

<p style="text-align: left; font-size: 16px; color:#131fcf"><span>🖥️  Ecrivez le code permettant de détecter les émotions dans le flux audio en provenance du microphone de votre ordinateur. </span></p>

<p style="text-align: left; font-size: 16px; color:#131fcf"><span>🖥️  Pour aller plus loin, utilisez la fonction de détection de silence pour arrêter l'acquisition audio lorsqu'il y a eu un silence pendant 5 secondes ou plus </span></p>